In [1]:
import numpy as np
import pandas as pd
import dill
import nltk.corpus

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer




In [2]:
df = pd.read_csv('train.csv')
display(df.head(), len(df) - df.count(), df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


id           0
title      558
author    1957
text        39
label        0
dtype: int64

None

In [3]:
values = {'title' : 'title','author' : 'author'}

In [4]:
df[['title', 'author']] = df[['title', 'author']].fillna(value = 'miss')

In [5]:
df.dropna(inplace=True)

In [6]:
len(df)-df.count()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
df.count(axis=0)

id        20761
title     20761
author    20761
text      20761
label     20761
dtype: int64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.2, random_state=42)

In [9]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [10]:
features = ['title', 'text']
target = 'label'

In [11]:
title = Pipeline([
                ('missing', TextTransformer('title', '')),
                ('selector', ColumnSelector(key='title')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10, stop_words='english'))
            ])
text = Pipeline([
                ('missing', TextTransformer('text', '')),
                ('selector', ColumnSelector(key='text')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10, stop_words='english'))
            ])


In [12]:
feats = FeatureUnion([('title', title),
                      ('text', text)])

In [13]:
%%time

pipeline = Pipeline([
    ('feats',feats),
    ('predict', GradientBoostingClassifier()),
])

pipeline.fit(X_train, y_train)

CPU times: user 2min 18s, sys: 2.63 s, total: 2min 20s
Wall time: 2min 24s


Pipeline(steps=[('feats',
                 FeatureUnion(transformer_list=[('title',
                                                 Pipeline(steps=[('missing',
                                                                  TextTransformer(key='title',
                                                                                  value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.9,
                                                                                  min_df=10,
                                                                                  stop_words='english'))])),
                                                ('text',
                                                

In [14]:
with open('pipeline_model.dill', 'wb') as f:
    dill.dump(pipeline, f)

In [17]:
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)